In [2]:
# This notebook :
# - adds the slip and area number to a csv file by searching through the id string
# - makes a new database with the coordinate values of all of the cells for all slip areas without repeats
# - adds the coordinate values to the time event database

In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from st_dbscan import ST_DBSCAN
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import Normalize

# import files
file_path_time = r"C:\Users\ainii\OneDrive\Documents\TempSpa\locations\sortedfiles\ST-DBSCAN\event_time_list\468.csv" # add file path
file_path_spatial = r"C:\Users\ainii\OneDrive\Documents\TempSpa\locations\sortedfiles\ST-DBSCAN\eventdata\L468_events.csv"
df_468_time = pd.read_csv(file_path_time)
df_468_spatial = pd.read_csv(file_path_spatial)


# print(df_468_time.head())
# print(df_468_spatial.head())

In [ ]:
# add column indicating slip area

# Define a function to extract slip and area number from the ID
def extract_slip_area(row):
    # Access the ID string from the first column
    id_str = row.iloc[0]  # Assuming the ID is always in the first column
    # Split the ID string into parts based on underscores
    parts = id_str.split('_')
    # Search for parts containing 'slip' and 'area'
    slip = next((part for part in parts if 'slip' in part), None)
    area = next((part for part in parts if 'area' in part), None)
    # Combine slip and area numbers if found
    if slip and area:
        return f"{slip}_{area}"
    else:
        return None

df_468_time['Slip_Area'] = df_468_time.apply(extract_slip_area, axis=1)

# Save the DataFrame back to the database
df_468_time.to_csv(file_path_time, index=False)  # Or any other method to save your DataFrame back to your database

print(df_468_time.head())

In [9]:
# extract coordinate information from events data

# Load spatial data into a pandas DataFrame
file_path_spatial = r"C:\Users\ainii\OneDrive\Documents\TempSpa\locations\sortedfiles\ST-DBSCAN\eventdata\L468_events.csv"
df_468_spatial = pd.read_csv(file_path_spatial)

# Group by cell characteristics (column 12 and column 13) and select the first occurrence of each cell
unique_cells_df = df_468_spatial.groupby(['Slip_Area', 'Cell_Number']).first().reset_index()

# Select the required columns
new_df = unique_cells_df[['cell_x', 'cell_y', 'Slip_Area', 'Cell_Number']]

# Define the path for the new database
new_file_path = r"C:\Users\ainii\OneDrive\Documents\TempSpa\locations\sortedfiles\ST-DBSCAN\temp_spatial_coordinates_data\L468_spatial_info.csv"

# Save the new DataFrame to the specified folder
new_df.to_csv(new_file_path, index=False)



In [10]:
# File paths
file_path_spatial_info = r"C:\Users\ainii\OneDrive\Documents\TempSpa\locations\sortedfiles\ST-DBSCAN\temp_spatial_coordinates_data\L468_spatial_info.csv"

# Read spatial information and event data
df_spatial_info = pd.read_csv(file_path_spatial_info)
df_event_data = pd.read_csv(file_path_time)

# Merge dataframes based on Slip_Area and Cell_Number
merged_df = pd.merge(df_event_data, df_spatial_info, on=['Slip_Area', 'Cell_Number'], how='left')

# Save the merged dataframe back to the event data file
merged_df.to_csv(file_path_time, index=False)
